### Code to create Grid Map with TimSlider
Timeslider covers one year, over weekly intervals
This was run locally, as installation of geopandas in the remote machine proved difficult

#### Import libraries

In [1]:
import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import geojson
import json
import datetime
from datetime import datetime
from datetime import timedelta

#### Prepare weekly region count dataframe

In [2]:
weekly_region_counts = pd.read_csv('weekly_region_counts.csv')
del weekly_region_counts['Unnamed: 0']
weekly_region_counts

,region,count,week
0,"(20,24)",2637,0
1,"(14,6)",1799,0
2,"(9,15)",974,0
3,"(12,15)",790,0
4,"(13,6)",781,0
...,...,...,...
19709,"(29,7)",1,49
19710,"(21,13)",1,49
19711,"(22,10)",1,49
19712,"(27,21)",1,49


#### Prepare geometry dataframe

In [3]:
lower_left = [20, -135] 
upper_right = [50, -60]

def get_geojson_grid(upper_right, lower_left, m, n):
    geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}
    
    lat_steps = np.linspace(lower_left[0], upper_right[0], m+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)
    
    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]
    
    lat_count = 0
    for lat in lat_steps[:-1]:
        lon_count = 0
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Identifier
            identifier = '(' + str(lat_count) + ',' + str(lon_count) + ')'
            
            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates]
                },
                "properties":{
                    "region": identifier
                },
            }

            geo_json["features"].append(grid_feature)
            lon_count+=1
            
        lat_count+=1
    return geo_json

grid = get_geojson_grid(upper_right, lower_left, 30, 30)
# print(grid["features"])
with open('grid.geojson', 'w') as f:
   geojson.dump(grid, f)

grid_df = gpd.read_file('grid.geojson')
grid_df

,region,geometry
0,"(0,0)","POLYGON ((-135.00000 21.00000, -132.50000 21.0..."
1,"(0,1)","POLYGON ((-132.50000 21.00000, -130.00000 21.0..."
2,"(0,2)","POLYGON ((-130.00000 21.00000, -127.50000 21.0..."
3,"(0,3)","POLYGON ((-127.50000 21.00000, -125.00000 21.0..."
4,"(0,4)","POLYGON ((-125.00000 21.00000, -122.50000 21.0..."
...,...,...
895,"(29,25)","POLYGON ((-72.50000 50.00000, -70.00000 50.000..."
896,"(29,26)","POLYGON ((-70.00000 50.00000, -67.50000 50.000..."
897,"(29,27)","POLYGON ((-67.50000 50.00000, -65.00000 50.000..."
898,"(29,28)","POLYGON ((-65.00000 50.00000, -62.50000 50.000..."


#### Merge count and geometry dataframe

In [4]:
total_weeks = 50
regioncount_geo_dfs = []
# recall week starts at week_index 0
for week in range(total_weeks):
    temp_df = weekly_region_counts[weekly_region_counts['week']==week]
    temp_df = pd.merge(temp_df,grid_df, how = 'left', on="region")
    temp_df['week']=week
    temp_df['count'] = temp_df['count'].fillna(0)
    regioncount_geo_dfs.append(temp_df)
regioncount_df_with_geometry = pd.concat(regioncount_geo_dfs)

#### Set opacity to 1

In [5]:
regioncount_df_with_geometry['opacity'] = 1
regioncount_df_with_geometry

,region,count,week,geometry,opacity
0,"(20,24)",2637,0,"POLYGON ((-75.00000 41.00000, -72.50000 41.000...",1
1,"(14,6)",1799,0,"POLYGON ((-120.00000 35.00000, -117.50000 35.0...",1
2,"(9,15)",974,0,"POLYGON ((-97.50000 30.00000, -95.00000 30.000...",1
3,"(12,15)",790,0,"POLYGON ((-97.50000 33.00000, -95.00000 33.000...",1
4,"(13,6)",781,0,"POLYGON ((-120.00000 34.00000, -117.50000 34.0...",1
...,...,...,...,...,...
379,"(29,7)",1,49,"POLYGON ((-117.50000 50.00000, -115.00000 50.0...",1
380,"(21,13)",1,49,"POLYGON ((-102.50000 42.00000, -100.00000 42.0...",1
381,"(22,10)",1,49,"POLYGON ((-110.00000 43.00000, -107.50000 43.0...",1
382,"(27,21)",1,49,"POLYGON ((-82.50000 48.00000, -80.00000 48.000...",1


#### Set colors based on count values

In [6]:
max_colour = max(regioncount_df_with_geometry['count'])
min_colour = min(regioncount_df_with_geometry['count'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
regioncount_df_with_geometry['colour'] = regioncount_df_with_geometry['count'].map(cmap)
regioncount_df_with_geometry

,region,count,week,geometry,opacity,colour
0,"(20,24)",2637,0,"POLYGON ((-75.00000 41.00000, -72.50000 41.000...",1,#fed26eff
1,"(14,6)",1799,0,"POLYGON ((-120.00000 35.00000, -117.50000 35.0...",1,#ffe38bff
2,"(9,15)",974,0,"POLYGON ((-97.50000 30.00000, -95.00000 30.000...",1,#fff1a8ff
3,"(12,15)",790,0,"POLYGON ((-97.50000 33.00000, -95.00000 33.000...",1,#fff4afff
4,"(13,6)",781,0,"POLYGON ((-120.00000 34.00000, -117.50000 34.0...",1,#fff4b0ff
...,...,...,...,...,...,...
379,"(29,7)",1,49,"POLYGON ((-117.50000 50.00000, -115.00000 50.0...",1,#ffffccff
380,"(21,13)",1,49,"POLYGON ((-102.50000 42.00000, -100.00000 42.0...",1,#ffffccff
381,"(22,10)",1,49,"POLYGON ((-110.00000 43.00000, -107.50000 43.0...",1,#ffffccff
382,"(27,21)",1,49,"POLYGON ((-82.50000 48.00000, -80.00000 48.000...",1,#ffffccff


#### Set date_sec based on week index

In [7]:
def get_date(week):
    delta = timedelta(weeks = week)
    week_time = datetime.strptime('2020-03-19', '%Y-%m-%d') + delta
    return int(week_time.timestamp())
regioncount_df_with_geometry['date_sec'] = regioncount_df_with_geometry['week'].apply(get_date)
regioncount_df_with_geometry

,region,count,week,geometry,opacity,colour,date_sec
0,"(20,24)",2637,0,"POLYGON ((-75.00000 41.00000, -72.50000 41.000...",1,#fed26eff,1584590400
1,"(14,6)",1799,0,"POLYGON ((-120.00000 35.00000, -117.50000 35.0...",1,#ffe38bff,1584590400
2,"(9,15)",974,0,"POLYGON ((-97.50000 30.00000, -95.00000 30.000...",1,#fff1a8ff,1584590400
3,"(12,15)",790,0,"POLYGON ((-97.50000 33.00000, -95.00000 33.000...",1,#fff4afff,1584590400
4,"(13,6)",781,0,"POLYGON ((-120.00000 34.00000, -117.50000 34.0...",1,#fff4b0ff,1584590400
...,...,...,...,...,...,...,...
379,"(29,7)",1,49,"POLYGON ((-117.50000 50.00000, -115.00000 50.0...",1,#ffffccff,1614229200
380,"(21,13)",1,49,"POLYGON ((-102.50000 42.00000, -100.00000 42.0...",1,#ffffccff,1614229200
381,"(22,10)",1,49,"POLYGON ((-110.00000 43.00000, -107.50000 43.0...",1,#ffffccff,1614229200
382,"(27,21)",1,49,"POLYGON ((-82.50000 48.00000, -80.00000 48.000...",1,#ffffccff,1614229200


#### Create Style Dictionary

In [8]:
region_list = regioncount_df_with_geometry['region'].unique().tolist()
region_idx = range(len(region_list))

style_dict = {}
for i in region_idx:
    region = region_list[i]
    result = regioncount_df_with_geometry[regioncount_df_with_geometry['region'] == region]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': r['opacity']}
    style_dict[str(i)] = inner_dict

#### Make a dataframe containing the features for each country

In [9]:
geo_df = regioncount_df_with_geometry[['geometry']]
geo_gdf = gpd.GeoDataFrame(geo_df)
geo_gdf = geo_gdf.drop_duplicates().reset_index()

#### Finally, we create our map and add a colourbar

In [10]:
from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron')

_ = TimeSliderChoropleth(
    data=geo_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Number of covid19-related tweets from an area"
slider_map.save(outfile='TimeSliderChoropleth.html')